<a href='https://www.darshan.ac.in/'> <img src='https://www.darshan.ac.in/Content/media/DU_Logo.svg' width="250" height="300"/></a>
<pre>
<center><b><h1>Data Mining</b></center>
<center><b><h1>Malay Panara | 23010101184 | 22/08/2025</b></center>
<pre>

# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

In [3]:
# Load the dataset
# Preprocess as per the instructions above | We have already done in TASK 2


# Your Code Here
# Step 1: Load the dataset
df = pd.read_csv("OnlineRetail.csv", encoding='unicode_escape')

# Step 2: Remove rows with missing values
df.dropna(inplace=True)

# Step 3: Keep only rows with Quantity > 0
df = df[df['Quantity'] > 0]

# for Basket
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [4]:
# Implement apriori functions below
def generate_candidates(prev_frequent_itemsets, k):
    """
    Generate candidate itemsets of size k from frequent itemsets of size k-1
    """
    
    candidates = []
    prev_list = list(prev_frequent_itemsets)
    
    for i in range(len(prev_list)):
        for j in range(i+1,len(prev_list)):
            union_set = prev_list[i].union(prev_list[j])
            if len(union_set) == k and union_set not in candidates:
                candidates.append(union_set)
    
    return candidates

def calculate_support(transactions, candidates):
    """
    Calculate support for each candidate in transactions
    Returns dictionary {itemset: support}
    """
    
    support_counts = {c:0 for c in candidates}
    num_transactions = len(transactions)
    
    for t in transactions:
        for c in candidates:
            if c.issubset(t):
                support_counts[c] += 1
    
    #Convert to support values
    support_data = {c : support_counts[c]/num_transactions for c in support_counts}
    return support_data

def get_frequent_itemsets(transactions, min_support):
    """
    Main Apriori frequent itemset generator
    """
    transactions = list(map(set,transactions))
    single_items = set(item for t in transactions for item in t)

    # First level: 1-itemsets
    candidates = [frozenset([i]) for i in single_items]
    support_data = calculate_support(transactions,candidates)
    frequent_itemsets = {frozenset(i) : s for i,s in support_data.items() if s >= min_support}
    
    all_frequent_itemsets = dict(frequent_itemsets)
    k = 2
    
    while frequent_itemsets:
        candidates = generate_candidates(frequent_itemsets.keys(),k)
        support_data = calculate_support(transactions,candidates)
        frequent_itemsets = {frozenset(i) : s for i,s in support_data.items() if s >= min_support}
        all_frequent_itemsets.update(frequent_itemsets)
        k += 1
    
    return all_frequent_itemsets

## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [5]:
# Function to generate rules from frequent itemsets

def generate_rules(frequent_itemsets, min_confidence):
    """
    Generate association rules from frequent itemsets
    """
    rules = []
    
    for itemset,support in frequent_itemsets.items():
        if len(itemset) >= 2:
            for i in range(1,len(itemset)):
                for antecedent in combinations(itemset,i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset - antecedent
                    if antecedent in frequent_itemsets:
                        confidence = support/frequent_itemsets[antecedent]
                        if confidence >= min_confidence:
                            rules.append((antecedent,consequent,support,confidence))

    return rules

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [6]:
# Output the final results
# Optional: Add visualizations

# Parameters
min_support = 0.02
min_confidence = 0.05

# Generate frequent itemsets and rules
transactions = basket.apply(lambda row: set(row[row==1].index), axis=1).tolist()
frequent_itemsets = get_frequent_itemsets(transactions, min_support)
rules = generate_rules(frequent_itemsets, min_confidence)

# Top 10 frequent itemsets
top_freq = sorted(frequent_itemsets.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 Frequent Itemsets:")
for itemset, support in top_freq:
    print(f"{set(itemset)}: support = {support:.2f}")

# Top 10 association rules
top_rules = sorted(rules, key=lambda x: x[3], reverse=True)[:10]
print("\nTop 10 Association Rules:")
for antecedent, consequent, support, confidence in top_rules:
    print(f"{set(antecedent)} -> {set(consequent)}, support = {support:.2f}, confidence = {confidence:.2f}")

Top 10 Frequent Itemsets:
{'WHITE HANGING HEART T-LIGHT HOLDER'}: support = 0.11
{'REGENCY CAKESTAND 3 TIER'}: support = 0.09
{'JUMBO BAG RED RETROSPOT'}: support = 0.09
{'PARTY BUNTING'}: support = 0.07
{'ASSORTED COLOUR BIRD ORNAMENT'}: support = 0.07
{'LUNCH BAG RED RETROSPOT'}: support = 0.07
{'SET OF 3 CAKE TINS PANTRY DESIGN '}: support = 0.06
{'POSTAGE'}: support = 0.06
{'LUNCH BAG  BLACK SKULL.'}: support = 0.06
{'PACK OF 72 RETROSPOT CAKE CASES'}: support = 0.06

Top 10 Association Rules:
{'PINK REGENCY TEACUP AND SAUCER', 'ROSES REGENCY TEACUP AND SAUCER '} -> {'GREEN REGENCY TEACUP AND SAUCER'}, support = 0.02, confidence = 0.89
{'PINK REGENCY TEACUP AND SAUCER', 'GREEN REGENCY TEACUP AND SAUCER'} -> {'ROSES REGENCY TEACUP AND SAUCER '}, support = 0.02, confidence = 0.85
{'PINK REGENCY TEACUP AND SAUCER'} -> {'GREEN REGENCY TEACUP AND SAUCER'}, support = 0.02, confidence = 0.83
{'PINK REGENCY TEACUP AND SAUCER'} -> {'ROSES REGENCY TEACUP AND SAUCER '}, support = 0.02, confid